In [11]:
import numpy as np
import pandas as pd
import scipy.sparse as sc

from sklearn.model_selection import train_test_split

In [12]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

# Données de tests complètes
test_inputs = df_test.iloc[:, :-1]
test_labels = df_test.iloc[:, -1:]

def split_training_data(df):
    train_set = []
    valid_set = []
    
    for (i,row) in enumerate(df):
        if i % 5 < 4:
            train_set.append(list(df))
        else:
            valid_set.append(list(df))
    
    return (np.array(train_set), np.array(valid_set))


# Données complètes
X_columns = [c for c in df_train.columns if c not in ['S.No', 'LABELS']]
train_inputs = df_train.loc[:, X_columns]
train_labels = df_train.loc[:, 'LABELS']

# On trie le training set
df_train_sorted = df_train.sort_values(by=['LABELS'], axis=0, ascending=False)
df_train_light = df_train_sorted.iloc[5000:13000, :].sample(frac=1).reset_index(drop=True)

# Données sélectionné un peu plus égal pour chaque classe pour 1 seule grande batch
train_inputs_light = df_train_light.iloc[:, :-1]
train_labels_light = df_train_light.iloc[:, -1:]


In [13]:
train_set, valid_set = train_split_training_data(df_train_light)

NameError: name 'train_split_training_data' is not defined

In [ ]:
# Permet l'encodage onehot de n'importe quel scalaire n_class est simplement le nombre de classe qui va
# donc nous donner la longueur que le vecteur final aura.
def onehot(x,n_class):
    encoded = []
    for i in x:
        yi_encoded = np.zeros(n_class)
        yi_encoded[int(i)] = 1
        encoded.append(yi_encoded)
    return np.array(encoded)

# Prends un vecteur et retourne un vecteur où a été appliqué la fonction softmax sur tous les éléments du vecteur x.
def softmax_vect(x):
    y = np.exp(x-np.max(x))
    div = np.sum(y)
    return y/div

# Retourne la proportion de bonne prédictions trouver avec un certain estimateur. y_pred et y_vrai ne doivent pas être
# encodé en onehot.
def accuracy(y_pred, y_vrai):
    somme = 0
    for i in range(len(y_vrai)):
        if np.array_equal(y_vrai[i],y_pred[i]):
            somme+=1
    return somme / len(y_vrai)

In [ ]:
class SoftmaxRegression:

    def __init__(self, y_labels, x_train):
        self.y_labels = y_labels
        self.x_train = x_train
        self.nb_classe = len(np.unique(y_labels))

    # Cette fonction va trouver le w et le b qui sont optimals.
    def train(self, eta, lam, nb_iterations):

        w_current = np.random.random([len(x_train[1, :]), 3])
        b_current = np.random.random(1)
        y_onehot = onehot(self.y_labels, self.nb_classe)

        for i in range(nb_iterations):
            z = np.dot(x_train, w_current) + b_current
            y_hat = np.array([softmax_vect(x) for x in z])
            w_grad = (1 / len(self.x_train[:, 0])) * np.dot(np.transpose(self.x_train), (y_hat - y_onehot)) + 2 * lam * w_current
            b_grad = (1 / len(self.x_train[:, 0])) * np.sum(y_hat - y_onehot)
            w_current = w_current - eta * w_grad
            b_current = b_current - eta * b_grad
            self.w = w_current
            self.b = b_current
        self.w = w_current
        self.b = b_current

    # Fonction qui retourne un vecteur contenant les classes de toutes les prédictions.
    def predict(self, x_test):
        z = np.dot(x_test, self.w) + self.b
        y_onehot = onehot(self.y_labels, self.nb_classe)
        y_hat = np.array([softmax_vect(x) for x in z])
        class_pred = [np.argmax(y_hat[i]) for i in range(len(y_hat))]
        return class_pred

In [ ]:
def test_model(modelclass, w0=[-3.0, 3.0, 0.1], reg=.1, stepsize=.2, plot=False):
    """Crée une instance de modelclass, entraîne la, calcule le taux d'erreurs sur un
    test set, trace les courbes d'apprentissage et la frontieres de decision.
    """
    model = modelclass(w0, reg)
    training_loss, training_error = model.train(trainset, stepsize, 100, plot=plot)
    print("The test error is {:.2f}%".format(
      model.test(testset[:,:-1], testset[:,-1])*100))
    print('Initial weights: ', w0)
    print('Final weights: ', model.w)
    print("Last train loss", training_loss.mean())

    # learning curves
    fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(8,2))
    ax0.plot(training_loss)
    ax0.set_title('loss')
    ax1.plot(training_error)
    ax1.set_title('error rate')

    # data plot
    plt.figure()
    scatter(trainset, marker='x')
    scatter(testset, marker='^')
    decision_boundary(model.w)
    finalize_plot(modelclass.__name__)